1 - Importación de librerías

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt #Ploting charts
from glob import glob #retriving an array of files in directories
from keras.models import Sequential #for neural network models
from keras.layers import Dense, Dropout, Flatten, ZeroPadding2D, Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator #Data augmentation and preprocessing
from keras.utils import to_categorical #For One-hot Encoding
from keras.optimizers import Adam, SGD, RMSprop #For Optimizing the Neural Network
from keras.callbacks import EarlyStopping

2 - Carga de documento a través de GDrive

In [ ]:
from google.colab import drive
drive.mount("/content/drive/", force_remount=True)
drive.mount('/content/drive/')

Mounted at /content/drive/
Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
!mkdir '/content/drive/MyDrive/U5M10/'


mkdir: cannot create directory ‘/content/drive/MyDrive/U5M10/’: File exists


In [ ]:
#!unzip archive.zip -d '/content/drive/MyDrive/U5M10'


import zipfile
with zipfile.ZipFile('/content/drive/MyDrive/U5M10/archive.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/drive/MyDrive/U5M10')

In [ ]:
import matplotlib.pyplot as plt
from keras.preprocessing import image
import numpy as np

In [ ]:
import os
paths = os.listdir(path='/content/drive/MyDrive/U5M10/chest_xray/chest_xray')
print(paths)

['.DS_Store', 'test', 'train', 'val']


In [ ]:
import pathlib

train_path= pathlib.Path('/content/drive/MyDrive/U5M10/chest_xray/chest_xray/train')
valid_path = pathlib.Path('/content/drive/MyDrive/U5M10/chest_xray/chest_xray/val')
test_path = pathlib.Path('/content/drive/MyDrive/U5M10/chest_xray/chest_xray/test')


In [ ]:
#Comprobamos que las carpetas se hayan cargado correctamente con todas sus imágenes
train_size = len(list(train_path.glob("*/*")))
valid_size = len(list(valid_path.glob("*/*")))
test_size = len(list(test_path.glob("*/*")))

print(f"Total images in train set\t: {train_size}")
print(f"Total images in valid set\t: {valid_size}")
print(f"Total imags in test set\t\t: {test_size}")

Total images in train set	: 5218
Total images in valid set	: 18
Total imags in test set		: 624


In [ ]:
from pathlib import Path


Procesamiento de imágenes para evitar overfitting

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255, # escalamos y obtener valores entre 0 y 1
        shear_range=0.2, # 20% angulo de rotación
        zoom_range=0.2, # data augmentation en un 20%, usa intervalo (0.8,1.2)
        rotation_range=20,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

training_dataset = train_datagen.flow_from_directory('/content/drive/MyDrive/U5M10/chest_xray/chest_xray/train',#carga de fotos
                                                    target_size=(64, 64),#reescalamos las fotos a la misma medida
                                                    batch_size=32, #número de imágenes a procesar, para cada 32 fotos se revisarán los pesos
                                                    class_mode='binary') #porque solo tenemos dos tipos de fotos

testing_dataset = test_datagen.flow_from_directory('/content/drive/MyDrive/U5M10/chest_xray/chest_xray/test',
                                                target_size=(64, 64),
                                                batch_size=32,
                                                class_mode='binary')


Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


3 - Construir el modelo de CNN

In [ ]:
classifier = Sequential() #Inicializamos con Sequential, para que nos cree los pesos de forma aleatoria.


In [ ]:
# creamos la primera capa de convolución la función Conv2D con sus argumentos
classifier.add(Conv2D(filters = 32, # numero de filtros a aplicar, multiplo de 2^5
                      kernel_size = (3, 3), # tamaño del filtro
                      input_shape = (64, 64, 3), # tamaño de entrada de las fotos y si es en color o no
                      activation = "relu") # función de activación
                      )


In [ ]:
#reducción de dimensión
classifier.add(MaxPooling2D(pool_size = (2,2)))


In [ ]:
#Aplicamos una segunda capa convolucional más max pooling.
classifier.add(Conv2D(filters = 64,kernel_size = (3, 3), activation = "relu"))
classifier.add(MaxPooling2D(pool_size = (2,2)))

In [ ]:
#preparación de la entrada a la red neuronal
classifier.add(Flatten()) #aplanamos la matriz en un tensor de una dimensión
classifier.add(Dropout(0.5)) #aleatoriamente apagamos neuronas y así ayudamos a controlar el overfitting


In [ ]:
#generamos la red con el método Dense
classifier.add(Dense(units = 128, activation = "relu"))
classifier.add(Dense(units = 1, activation = "sigmoid")) #utilizamos sigmoid porque tenemos dos categorías (pneumenía/normal)


In [ ]:
#CALCULO DE LOS PARAMETROS

classifier.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 12544)             0         
                                                                 
 dropout (Dropout)           (None, 12544)             0

4 - Compilar y entrenar la CNN

In [ ]:
classifier.compile(optimizer = "RMSprop", 
                   loss = "binary_crossentropy", 
                   metrics = ["accuracy"])


In [ ]:
history = classifier.fit(training_dataset,
                        steps_per_epoch = training_dataset.n//32,#41744/32=250 (valor del batch_size),
                        epochs=5,
                        validation_data=testing_dataset,
                        validation_steps=testing_dataset.n//32 #624/32=19.5
                        )


Epoch 1/5
163/163 [==============================] - 129s 771ms/step - loss: 0.4692 - accuracy: 0.7926 - val_loss: 0.3131 - val_accuracy: 0.8602
Epoch 2/5
163/163 [==============================] - 117s 720ms/step - loss: 0.2999 - accuracy: 0.8754 - val_loss: 0.3886 - val_accuracy: 0.8109
Epoch 3/5
163/163 [==============================] - 124s 761ms/step - loss: 0.2633 - accuracy: 0.8909 - val_loss: 0.2475 - val_accuracy: 0.8980
Epoch 4/5
 42/163 [======>.......................] - ETA: 1:30 - loss: 0.2437 - accuracy: 0.8996

5 - Validación del modelo

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
print('Avg. Train Accuracy: ',round(np.mean(acc),3)*100)
print('Avg. Test Accuracy:  ',round(np.mean(val_acc),4)*100)
print('Avg. Train Loss: ',round(np.mean(loss),3)*100)
print('Avg. Test Loss:  ',round(np.mean(val_loss),4)*100)
epochs = range(len(acc))

plt.figure()

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation acc')
plt.legend()
plt.show()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()


In [ ]:
classifier.save('pneumonia_normal.h5')


6 - Nuevas predicciones

In [ ]:

# import required module
import os
# assign directory
directory = '/content/drive/MyDrive/U5M10/chest_xray/val/NORMAL'
 
# iterate over files in
# that directory
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)

        # import required module
import os
# assign directory
directory = '/content/drive/MyDrive/U5M10/chest_xray/val/PNEUMONIA'
 
# iterate over files in
# that directory
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)

In [ ]:
import numpy as np
import keras
import keras.utils as image
#from keras.preprocessing import image
directory = '/content/drive/MyDrive/U5M10/chest_xray/val/NORMAL'



for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
      photo=f
    print(photo)
    test_image = keras.utils.load_img(photo, target_size = (64, 64, 3))
    test_image = image.img_to_array(test_image)
    print(test_image.shape)
    test_image1 = np.expand_dims(test_image, axis = 0)
    print(test_image1.shape)
    result = classifier.predict(test_image1)
    #print(result)
    training_dataset.class_indices
    if result[0] > np.mean(val_acc):
        prediction = 'PNEUMONIA'
    else:
        prediction = 'NORMAL'

    print('La foto es de un:' ,prediction)

    



 


In [ ]:
import numpy as np
import keras
import keras.utils as image
#from keras.preprocessing import image

directory = '/content/drive/MyDrive/U5M10/chest_xray/val/PNEUMONIA'


for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
      photo=f
    print(photo)
    test_image = keras.utils.load_img(photo, target_size = (64, 64, 3))
    test_image = image.img_to_array(test_image)
    print(test_image.shape)
    test_image1 = np.expand_dims(test_image, axis = 0)
    print(test_image1.shape)
    result = classifier.predict(test_image1)
    #print(result)
    training_dataset.class_indices
    if result[0] > np.mean(val_acc):
        prediction = 'PNEUMONIA'
    else:
        prediction = 'NORMAL'

    print('La foto es de un:' ,prediction)
